In [ ]:
#maria ahmed
#i202451

In [7]:
!pip install transformers
!pip install torch
!pip install datasets


In [8]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
from datasets import load_dataset

dataset = load_dataset("ccdv/pubmed-summarization")
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['article', 'abstract'],
        num_rows: 119924
    })
    validation: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6633
    })
    test: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6658
    })
})


In [13]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from datasets import load_metric
import string
import re

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


def preprocess_function(examples):
    inputs = examples["article"]
    targets = examples["abstract"]

    inputs = inputs.translate(str.maketrans("", "", string.punctuation))  
    inputs = inputs.lower()  

    targets = targets.translate(str.maketrans("", "", string.punctuation))  
    targets = targets.lower()  

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=["article", "abstract"])


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
!pip install rouge_score

In [15]:
!pip install accelerate -U

In [23]:
import transformers
import accelerate

print(transformers.__version__)
print(accelerate.__version__)


4.41.2
0.31.0


In [24]:
rouge_metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Replace -100 in labels as we can't decode them
    decoded_labels = [[label if label != -100 else tokenizer.pad_token_id for label in labels_seq] for labels_seq in labels]
    decoded_labels = tokenizer.batch_decode(decoded_labels, skip_special_tokens=True)

    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    return result

training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    eval_strategy="epoch",
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

trainer.train()

model.save_pretrained("fine-tuned-t5-small-pubmed")
tokenizer.save_pretrained("fine-tuned-t5-small-pubmed")


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`